In [75]:
import pandas as pd
import numpy as np
from dateutil.parser import parse
import math

# Clustering packages
import py_stringclustering as scl
import py_stringmatching as sm
import py_stringsimjoin as ssj

In [76]:
file_ = 'merged_data.csv'
df = pd.read_csv(file_, encoding='UTF-8')
del df['Unnamed: 0']


In [20]:
# df.columns

In [21]:
df['ResultCommentText'].dtype

dtype('O')

In [22]:
################# Function to check if the data type of a value is date ########################


true_date=0
false_date=0
nan_count_date = 0

def is_date(string):
    global true_date,false_date,nan_count_date
    if isinstance(string,float) and np.isnan(string):
        nan_count_date = nan_count_date+1
        # do nothing
    elif isinstance(string,float) or isinstance(string,int) :
        false_date=false_date+1
    else:
        try:
            parse(string)
            true_date=true_date+1
        except ValueError:
            false_date=false_date+1  

def is_date_column(col):
    global true_date,false_date,nan_count_date
    true_date=0
    false_date=0
    nan_count_date = 0

    col.apply(lambda x: is_date(x))
    if true_date > 0 and false_date == 0:
        return True
    else:
        return False
# true_date, false_date,nan_count_date
is_date_column(df['ActivityStartDate'])

True

In [23]:
################# Function to check if the data type of a value is numeric ########################


true_numeric=0
false_numeric=0
nan_count_numeric=0

def is_number(n):
    global true_numeric,false_numeric,nan_count_numeric
    if (isinstance(n,float) and np.isnan(n)):
        nan_count_numeric=nan_count_numeric+1
    else:
        try:
            float(n)   # Type-casting the string to `float`.
                       # If string is not a valid `float`, 
                       # it'll raise `ValueError` exception
            true_numeric=true_numeric+1
        except ValueError:
            false_numeric=false_numeric+1
def get_numeric_stats(col):
    global true_numeric,false_numeric,nan_count_numeric
    true_numeric=0
    false_numeric=0
    nan_count_numeric=0

    col.apply(lambda x: is_number(x))
    return true_numeric,false_numeric

get_numeric_stats(df['NormalizedResultMeasureValue'])
# true_numeric, false_numeric,nan_count_numeric

(1795, 4)

In [24]:
################# Function to check if the data type of a value is text ########################

true_text=0
false_text=0
nan_count_text=0

def is_text(text):
    global true_text,false_text,nan_count_text
    if (isinstance(text,float) and np.isnan(text)):
        nan_count_text=nan_count_text+1
    else:
        if (len(text) > 50):
            true_text=true_text+1
            # return True
        else:
            false_text=false_text+1
            # return False
            
def is_text_column(col):
    global true_text,false_text,nan_count_text
    true_text=0
    false_text=0
    nan_count_text=0

    col.apply(lambda x: is_text(x))
    if true_text>0 and false_text == 0:
        return True
    else:
        return False

true_text, false_text, nan_count_text
is_text_column(df['ResultCommentText'])


False

In [34]:
################# Function to check if the data type of a value is categorical ########################

true_categorical=0
false_categorical=0
nan_count_categorical=0

def is_categorical(col):
    global true_categorical,false_categorical,nan_count_categorical 
    true_categorical=0
    false_categorical=0
    nan_count_categorical=0
    
    length = len(col.unique())
    if length > 1 and length < 5:
        print (col.unique())
    return ( length > 1 and length < 5)
        
is_categorical(df['ResultCommentText'])

False

In [35]:
file_ = 'merged_data.csv'
df = pd.read_csv(file_, encoding='UTF-8')
del df['Unnamed: 0']
datatypes = {}
mixed_datatypes = []

for column in df:
    print (column)
    if is_date_column(df[column]): #true/false
        datatypes[column] = "DateTime"
    else:
        numeric_count, not_numeric_count = get_numeric_stats(df[column])
        if numeric_count == 0 and not_numeric_count == 0:
            datatypes[column] = "Null" 
        elif not_numeric_count == 0:
            datatypes[column] = "Numeric"
        elif numeric_count == 0:
            if is_text_column(df[column]): #true/false
                datatypes[column] = "Text"
            elif is_categorical(df[column]) :
                datatypes[column] = "Categorical"
            else:
                datatypes[column] = "String"
        elif not_numeric_count > 0 and numeric_count > 0:
            datatypes[column] = "Mixed"
            mixed_dt = {}
            mixed_dt["column"] = column
            mixed_dt["numeric_count"] = numeric_count
            mixed_dt["string_count"] = not_numeric_count
            mixed_datatypes.append(mixed_dt)
        else:
            print (column)
            


OrganizationIdentifier
['WIDNR_WQX' 'IL_EPA' 'IL_EPA_WQX' 'MNPCA']
OrganizationFormalName
['Wisconsin Department of Natural Resources' 'Illinois EPA' 'illinois epa'
 'Minnesota Pollution Control Agency - Ambient Surface Water']
ActivityIdentifier
ActivityTypeCode
['Field Msr/Obs' 'Sample-Routine' 'Sample']
ActivityMediaName
['Water' 'Sediment']
ActivityMediaSubdivisionName
[nan 'Surface Water']
ActivityStartDate
ActivityStartTime/Time
ActivityStartTime/TimeZoneCode
['CDT' 'CST' nan 'UTC']
ActivityEndDate
ActivityEndTime/Time
ActivityEndTime/TimeZoneCode
[nan 'CDT' 'CST']
ActivityDepthHeightMeasure/MeasureValue
ActivityDepthHeightMeasure/MeasureUnitCode
[nan 'ft' 'm']
ActivityDepthAltitudeReferencePointText
[nan 'SURFACE']
ActivityTopDepthHeightMeasure/MeasureValue
ActivityTopDepthHeightMeasure/MeasureUnitCode
[nan 'm']
ActivityBottomDepthHeightMeasure/MeasureValue
ActivityBottomDepthHeightMeasure/MeasureUnitCode
[nan 'm']
ProjectIdentifier
ActivityConductingOrganizationText
['WIDNR_WQX

In [31]:
print (datatypes)

{'OrganizationIdentifier': 'Categorical', 'OrganizationFormalName': 'Categorical', 'ActivityIdentifier': 'String', 'ActivityTypeCode': 'Categorical', 'ActivityMediaName': 'Categorical', 'ActivityMediaSubdivisionName': 'Categorical', 'ActivityStartDate': 'DateTime', 'ActivityStartTime/Time': 'DateTime', 'ActivityStartTime/TimeZoneCode': 'Categorical', 'ActivityEndDate': 'DateTime', 'ActivityEndTime/Time': 'DateTime', 'ActivityEndTime/TimeZoneCode': 'Categorical', 'ActivityDepthHeightMeasure/MeasureValue': 'Numeric', 'ActivityDepthHeightMeasure/MeasureUnitCode': 'Categorical', 'ActivityDepthAltitudeReferencePointText': 'Categorical', 'ActivityTopDepthHeightMeasure/MeasureValue': 'Numeric', 'ActivityTopDepthHeightMeasure/MeasureUnitCode': 'Categorical', 'ActivityBottomDepthHeightMeasure/MeasureValue': 'Numeric', 'ActivityBottomDepthHeightMeasure/MeasureUnitCode': 'Categorical', 'ProjectIdentifier': 'String', 'ActivityConductingOrganizationText': 'Categorical', 'MonitoringLocationIdentifie

In [68]:
len(df.columns)

71

In [72]:
#df['id'] = range(0, len(df))
# df = df.reset_index()

# df_2 = scl.read_data('merged_data_v2.csv')

In [105]:
df['NormalizedResultMeasureValue'].iloc[1319]
#sub_df.iloc[0]
df_col = pd.DataFrame(df['ResultAnalyticalMethod/MethodIdentifier'])
df_col.columns = ['name']
df_col['id'] = range(0, len(df_col))

In [106]:
# Block using Jaccard join with jacc3gr(s1, s2) >= 0.3
# Returns a DataFrame containing pairs of string IDs that satisfy the blocking condition
trigramtok = sm.QgramTokenizer(qval=3)
blocked_pairs = ssj.jaccard_join(df_col, df_col, 'id', 'id', 'name', 'name', trigramtok, 0.3)
blocked_pairs

0%                          100%
[##############################] | ETA: 00:00:01 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:02 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:01 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00 | ETA: 00:00:00
Total time elapsed: 00:00:02


,_id,l_id,r_id,_sim_score
0,0,31,31,1.0
1,1,47,31,1.0
2,2,49,31,1.0
3,3,86,31,1.0
4,4,105,31,1.0
5,5,126,31,1.0
6,6,144,31,1.0
7,7,166,31,1.0
8,8,183,31,1.0
9,9,200,31,1.0


In [107]:
# Define clustering similarity measure
jaccsim = sm.Jaccard()

# Calculate sim scores
# Returns a list of triplets of the form (id1, id2, sim)
sim_scores = scl.get_sim_scores(df_col, blocked_pairs, trigramtok, jaccsim)

sim_scores[:20]

[(32, 46, 1.0),
 (31, 47, 1.0),
 (32, 48, 1.0),
 (46, 48, 1.0),
 (31, 49, 1.0),
 (47, 49, 1.0),
 (32, 66, 1.0),
 (46, 66, 1.0),
 (48, 66, 1.0),
 (32, 85, 1.0),
 (46, 85, 1.0),
 (48, 85, 1.0),
 (66, 85, 1.0),
 (31, 86, 1.0),
 (47, 86, 1.0),
 (49, 86, 1.0),
 (31, 105, 1.0),
 (47, 105, 1.0),
 (49, 105, 1.0),
 (86, 105, 1.0)]

In [108]:
sim_matrix = scl.get_sim_matrix(df_col, sim_scores)

In [109]:
sim_matrix

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [113]:
from sklearn.cluster import AgglomerativeClustering
aggcl = AgglomerativeClustering(n_clusters=15, affinity='precomputed', linkage='complete')

### Returns a list of cluster labels
labels = aggcl.fit_predict(sim_matrix)

In [114]:
### Returns a list of clusters where each cluster is a list of strings
str_clusters = scl.get_clusters(df_col, labels)

In [115]:
str_clusters

[[nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  '445',
  '365.1',
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  '365.1',
  '445',
  '365.1',
  '445',
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  '365.1',
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  '365.1',
  '445',
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  '445',
  '365.1',
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  '365.1',
  '445',
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
  nan,
 